# Study Summary
Analysis of a data set from US dept of labor Bureau of Labor Statistics to reveal undelying connection between gender, race, and physical labor. This study has 3 major findings:  
1.   There is an inverse relationship between the percentage representation of women in an industry, and that industries liklihood to be classified as "physical labor"  
2.   The percentage of Whites, Blacks, Hispanics, Asians, and the total number of people employed in an industry are irrelevant in predicting if the industry will be classified as “physical labor”.  
3.   Women are more likely to work in industries with larger populations.  


In [6]:
import pandas
from pdb import set_trace as bp
from numpy import log
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
lm      = smf.ols
BLS_CSV = "./data/bls/bls.csv"

def bptest(olsModelData):
    """
    Python re-implementation of R Breusch Pagan test of homoskedasticity 'bptest()'
    @param olsModelData : linear model resulting from statsmodels.formula.api.ols call
    @return res         : Wrapper instance around statsmodels.stats.api.het_breuschpagan call
    """
    # Define toString value for bptest result
    class Bpresult:
        """
        Wrapper class around statsmodels.stats.api.het_breuschpagan
          .fieldnames: String fieldnames of Breusch Pagan test results
          .bpvalues  : Values corresponding to fieldnames
          .result    : Dictionary of combined fieldnames and values
          __str__()  : Overload ToString method for nice printing
        """
        def __init__(self, olsModelData):
            """
            Initialization of Bpresult wrapper
            @param olsModelData: linear model resulting from statsmodels.formula.api.ols call
            """
            self.olsModelData = olsModelData
            self.fieldnames   = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'p-value']
            self.bpvalues     = sms.het_breuschpagan(olsModelData.resid, olsModelData.model.exog)
            self.result = {}
            for i in range(len(self.fieldnames)):
                key   = self.fieldnames[i]
                value = self.bpvalues[i]
                self.result[key] = value
        def __str__(self):
            """
            Bpresult wrapper __str__ override
            Will print model formula, Lagrange multiplier statistic (BP), df_model value, and p-value
            """
            rep = '\n\t-- studentized Breusch-Pagan test --\n'
            rep += f"formula = {self.olsModelData.model.formula}\n"
            rep += f"BP      = {self.result['Lagrange multiplier statistic']}\n"
            rep += f"df      = {self.olsModelData.df_model}\n"
            if (self.result['p-value'] < (2.2 * (10**-16))):
                rep += f"p-value < 2.2e-16\n"
            else:
                rep += f"p-value = {self.result['p-value']}\n"
            rep +='\n'
            return rep
    res = Bpresult(olsModelData)
    return res

## Table fields
|    Field     | Description |
|--------------|-------------|
|   phy_lbr:   | Boolean is profession "Physical Labor" |
|   tot_emp:   | Total employed(Thousands) persons in that profession |
|   women:	   | Women(%) in that profession |
|   white:	   | White(%) in that profession |
|   black:	   | Black(%) in that profession |
|   asian:	   | Asian(%) in that profession |
|   hispanic:   | Hispanic(%) in that profession 

In [7]:
bls_df = pandas.read_csv(BLS_CSV)
print(bls_df.describe())

          phy_lbr       tot_emp       women       white       black  \
count  274.000000    274.000000  274.000000  274.000000  274.000000   
mean     0.259124   1737.175182   39.860584   79.359854   11.264599   
std      0.438956   3937.599421   20.054595    7.868102    5.857961   
min      0.000000     51.000000    5.900000   42.600000    0.700000   
25%      0.000000    150.000000   24.400000   74.500000    7.400000   
50%      0.000000    413.000000   36.450000   79.100000   10.250000   
75%      1.000000   1312.500000   52.900000   85.000000   14.175000   
max      1.000000  35043.000000   93.800000   96.700000   35.600000   

            asian    hispanic  
count  274.000000  274.000000  
mean     6.049635   16.681022  
std      4.905064    7.458869  
min      0.000000    3.800000  
25%      3.125000   11.625000  
50%      5.300000   15.050000  
75%      7.400000   19.975000  
max     47.200000   47.100000  


# Model 1: M1<br>  
In M1 weather or not a profession involves physical labor; `phy_lbr(i)` is a function of some weighted combination of all the other demographic information <br>  
Using a linear model to get the OLS estimates of the parameters of the following regression model, where `u(i)` is the regression error:<br>   

$$
\text{phy\_lbr}(i) = \beta_0 + \beta_1 \ln(\text{tot\_emp}(i)) + \beta_2 \text{women}(i) + \beta_3 \text{white}(i) + \beta_4 \text{black}(i) + \beta_5 \text{asian}(i) + \beta_6 \text{hispanic}(i) + u(i)
$$

In [8]:
bls_ols_phylbr = lm(formula = "phy_lbr ~ log(tot_emp) + women + white + black + asian + hispanic", data=bls_df).fit()
print(bls_ols_phylbr.summary())

                            OLS Regression Results                            
Dep. Variable:                phy_lbr   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     10.85
Date:                Sat, 21 Sep 2024   Prob (F-statistic):           8.28e-11
Time:                        22:35:01   Log-Likelihood:                -132.79
No. Observations:                 274   AIC:                             279.6
Df Residuals:                     267   BIC:                             304.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.5629      1.755      0.890   

# Major Finding

1.   There is an inverse relationship between the percentage representation of women in an industry, and that industries liklihood to be classified as "physical labor" <br>

In [ ]:
# F test: INSIGNIFICANT VARIABLES: tot_emp white black asian and hispanic
#ftest_1 <- linearHypothesis(bls_ols_phylbr, c("log(tot_emp)=0", "white=0", "black=0", "asian=0", "hispanic=0"))
#TODO

In [9]:
# Homoskedacicity BP test
print(bptest(bls_ols_phylbr))


	-- studentized Breusch-Pagan test --
formula = phy_lbr ~ log(tot_emp) + women + white + black + asian + hispanic
BP      = 89.90313287819804
df      = 6.0
p-value < 2.2e-16




In [ ]:
# T-Test of coefficients: heteroskedasticity-corrected
#TODO

# Model 2: M2<br>  

In M2 the percentage representation of women in a profession; `women(i)` is a function of some weighted combination of all the other demographic information.<br>  
Using a linear model to get the OLS estimates of the parameters of the following regression model, where `u(i)` is the regression error:<br>   

$$
\text{women}(i) = \beta_0 + \beta_1 \ln(\text{tot\_emp}(i)) + \beta_2 \text{phy\_lbr}(i) + \beta_3 \text{white}(i) + \beta_4 \text{black}(i) + \beta_5 \text{asian}(i) + \beta_6 \text{hispanic}(i) + u(i)
$$

In [10]:
bls_ols_women = lm(formula = "women ~ log(tot_emp) + phy_lbr + white + black + asian + hispanic", data=bls_df).fit()
print(bls_ols_women.summary())

                            OLS Regression Results                            
Dep. Variable:                  women   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     19.28
Date:                Sat, 21 Sep 2024   Prob (F-statistic):           1.16e-18
Time:                        22:37:34   Log-Likelihood:                -1160.5
No. Observations:                 274   AIC:                             2335.
Df Residuals:                     267   BIC:                             2360.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      120.2058     74.463      1.614   

# Major Finding

2.   The percentage of Whites, Blacks, Hispanics, Asians, and the total number of people employed in an industry are irrelevant in predicting if the industry will be classified as “physical labor”.  
3.   Women are more likely to work in industries with larger populations.  

In [ ]:
# F test: asain=1
#ftest_2 <- linearHypothesis(bls_ols_phylbr, c("asian=1"))
#TODO

In [11]:
print(bptest(bls_ols_women))


	-- studentized Breusch-Pagan test --
formula = women ~ log(tot_emp) + phy_lbr + white + black + asian + hispanic
BP      = 27.40134344289849
df      = 6.0
p-value = 8.103436888723292e-05




In [ ]:
# T-Test of coefficients: heteroskedasticity-corrected
#TODO

In [ ]:
#################################################################################################################
# Extra fun stuff :)
#################################################################################################################
# M1: Ignoring insignificant variables
# Exasterbate M1 findings
#bls_ols_phylbr_clean = lm(formula = "phy_lbr ~ women", data=bls_df).fit()
#print(bls_ols_phylbr_clean.summary())

# M2: Ignoring insignificant variables
# Exasterbate M2 findings
#bls_ols_women_clean = lm(formula = "women ~ log(tot_emp) + phy_lbr + white + black + asian", data=bls_df).fit()
#print(bls_ols_phylbr_clean.summary())

# Actual numbers of things
# women variable is % of workforce that is women
# tot_emp is the number of people in the workforce
# women * tot_emp is the number of WOMEN in the workforce
#bls_ols_num = lm(formula = "I(women*tot_emp) ~ phy_lbr + I(white*tot_emp) + I(black*tot_emp) + I(asian*tot_emp) + I(hispanic*tot_emp)", data = bls_df).fit()
#bls_ols_num = lm(formula = "phy_lbr ~ I(women*tot_emp) + I(white*tot_emp) + I(black*tot_emp) + I(asian*tot_emp) + I(hispanic*tot_emp)", data = bls_df).fit()
#print(bls_ols_phylbr_clean.summary())

# Percent increase in population as a function of demographics
# Which demographics are indicators of labor force growth
#bls_ols_totemp = lm(formula = "log(tot_emp) ~ phy_lbr + women + white + black + asian + hispanic", data = bls_df).fit()
#print(bls_ols_totemp.summary())